In [1]:
from src.model.llm import GraphLLM
import torch
import os
from torch_geometric.loader import DataLoader
from src.dataset.obqa import OBQADataset
import json
from tqdm.notebook import tqdm

In [2]:
cpnet_dir = '/SSL_NAS/concrete/data/cpnet'
with open(os.path.join(cpnet_dir,'concept.txt'), 'r') as f:
    cpnet_txt = f.readlines()

In [3]:
device='cuda:3'
cpnet_dir = '/SSL_NAS/concrete/data/cpnet'
text_dir = '/SSL_NAS/concrete/data/obqa/statement'
graph_dir = '/SSL_NAS/concrete/data/obqa/graph'

In [4]:
phase = 'test'
infer_dataset = OBQADataset(
    data_path=os.path.join(text_dir, f'{phase}.jsonl'),
    graph_path=os.path.join(graph_dir, f'{phase}.graph.adj.pk-nodenum200.loaded_cache'),
    init_emb_path=os.path.join(cpnet_dir, 'tzw.ent.npy'),
)
infer_loader = DataLoader(infer_dataset, batch_size=1, shuffle=False)
len(infer_dataset)

500

In [11]:
question_id = 100
print(infer_dataset[question_id]['question'])
for idx, concept_id in enumerate(infer_dataset[question_id]['graphs'].concept_ids):
    if concept_id.item() in [0,1]:
        continue
    print((idx//200)+1,cpnet_txt[concept_id][:-1])

Choose the correct answer from the four options given.
Question: Grey clouds can bring
Options:
(a) sunlight
(b) falling water molecules
(c) blooming flowers
(d) drought conditions
Answer:

1 broachable
1 blue_sky
1 coastal_areas_of_alaska
1 grey_collar
1 grey_clouds
1 grey_fog
1 sunny_place
1 nimbyism
1 waterbed_hose
1 club
1 noctilucin
1 gas_form_of_water
1 cloudless
1 green
1 invisibly
1 blurry
1 deep_copy
1 clean_air
1 thunder
1 climate
1 fold
1 bills
1 sunny
1 sun
1 bright
1 yellow
1 black_white
1 polyester
1 hide_sun
1 moving_part
1 haze
1 bright_sun
1 earth
1 mars
1 lightsome
1 goat
1 bright_color
1 skies_color
1 gold_star_mother
1 cold_air
1 find_inside
1 could
1 trail
1 flugelman
2 broachable
2 blue_sky
2 coastal_areas_of_alaska
2 grey_collar
2 grey_clouds
2 grey_fog
2 water_fire
2 fall_down
2 ladder
2 atr
2 famil
2 bank_account
2 water_oak
2 water_particles
2 waterbed_hose
2 cooler
2 one_piece_of_medical_equipment
2 air_guitaring
2 condensed_matter
2 slurpee
2 surface_normal


In [5]:
class Args: pass
args = Args()
args.max_txt_len = 512
args.max_new_tokens = 32
args.llm_model_path = "google/flan-t5-xl"
args.llm_frozen = True
args.cache_dir = "/SSL_NAS/peoples/bonbak/model"
args.learning_rate = 1e-4
args.input_dim = 1024
args.hidden_dim = 1024
args.seed = 21

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(args.llm_model_path, cache_dir = args.cache_dir).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(args.llm_model_path, cache_dir = args.cache_dir)
word_embedding = model.get_input_embeddings()

In [8]:
model.eval()

prediction = []
ground_truth = []
model.eval()
with torch.no_grad():
    for samples in tqdm(infer_loader, total=len(infer_loader)):
        questions = tokenizer(samples['question'], add_special_tokens=False)
        inputs_embeds = word_embedding(torch.tensor(questions.input_ids).to(device))

        attention_mask = torch.tensor(questions.attention_mask).unsqueeze(dim=0).to(device)
        outputs = model.generate(
            inputs_embeds=inputs_embeds,
            max_new_tokens=args.max_new_tokens,
            attention_mask=attention_mask,
        )
        pred = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        gt = samples['answer']

        prediction.append(pred[0])
        ground_truth.append(gt[0])

  0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
num_match = 0
for pred, gt in zip(prediction, ground_truth):
    p = pred.replace('(','').replace(')','')[0].lower()
    a = gt.replace('(','').replace(')','')[0].lower()
    if p == a:
        num_match += 1
num_match / len(prediction)

0.734